In [1]:
import pandas as pd
import numpy as np
import system_prompts 
import human_prompts 
import utils

In [2]:
test_df = pd.read_excel("./gsl_eval_v0_all_except_llm.xlsx")

In [3]:
test_df = test_df[test_df[utils.GT_FIELD] != "not_sure"].reset_index(drop=True)
test_df.fillna("", inplace=True)

test_df["ori_title"] = np.where(test_df["listingTitle"] != "", test_df["listingTitle"], test_df["listingTitleEn"])
test_df["en_title"] = np.where(test_df["listingTitleEn"] != "", test_df["listingTitleEn"], test_df["listingTitle"])

In [5]:
test_df.queryEntities[0]

'tangibleItem:favors;style:nautical;occasion:baby shower;'

## Get additional query segment features

In [ ]:
sql_query = """
    qlm AS (
      select query_raw query, bin as queryBin 
      from `etsy-batchjobs-prod.snapshots.query_level_metrics_raw`
      where _date = date('2025-06-26')
    ),
    qis AS (
      SELECT query_raw query,
      CASE 
        WHEN class_id = 0 THEN 'broad' 
        WHEN class_id = 1 THEN 'direct_unspecified'
        WHEN class_id = 2 THEN 'direct_specified' 
      END AS qisClass
      FROM `etsy-search-ml-prod.mission_understanding.qis_scores`
    ),
    qisv2 AS (
      SELECT query,
      CASE 
        WHEN prediction = 0 THEN 'broad' 
        WHEN prediction = 1 THEN 'direct_unspecified'
        WHEN prediction = 2 THEN 'direct_specified' 
      END AS qisClassV2
      FROM `etsy-search-ml-prod.mission_understanding.qis_scores_v2`
    ),
    qe_new AS (
      SELECT 
      FROM `etsy-search-ml-prod.mission_understanding.query_entity_extraction_term_weighting`
    ),
"""


In [ ]:
input_texts = []
full_texts = []

for i in range(len(input_records)):
    human_prompt_text = utils.build_json_string(
        input_records[i],
        query_feature_dict=utils.query_feature_dict_options["query"],
        listing_feature_dict=utils.listing_feature_dict_options["tsidn"],
    )
    input_convo = [
        {"role": "system", "content": system_prompts.system_ethan_v2_noreason},
        {"role": "user", "content": human_prompt_text},
    ]
    full_convo = input_convo + [{"role": "assistant", "content": human_labels[i]}]
    input_text = tokenizer.apply_chat_template(
        input_convo,
        tokenize=False,
        add_generation_prompt=True
    )
    full_text = tokenizer.apply_chat_template(
        full_convo,
        tokenize=False,
        add_generation_prompt=False
    )
    input_texts.append(input_text)
    full_texts.append(full_text)
    
test_df['input_text'] = input_texts
test_df['full_text'] = full_texts